![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

# Pandas Report Generation Exercises

In [ ]:
# Read Files Data
import os
import pandas as pd
import xlsxwriter
from pathlib import Path

def init_df(file_name:str = 'tu_archivo.xlsx', sheet = None, init_row:int = 0):
    '''Lee un archivo xlsx y regresa un dataframe de pandas'''
    file_path = os.path.join(os.getcwd(),'data',file_name) # yo uso la carpeta data, pero puedes establecer tu propia ruta
    df = pd.read_excel(io=file_path,sheet_name=sheet, header = init_row)
    return df


In [59]:
df_base_fiscal = init_df(file_name='reporte ejecutivo clean.xlsx', sheet= 'Base Fiscal', init_row=3)
df_base_fiscal.head(5)

,uuid,version,rfcEmisor,nombreEmisor,regimenEmisor,rfcReceptor,nombreReceptor,regimenReceptor,fechaEmision,folio,...,004-D005-APORT EMPL FONDO DE AHORRO,004-D009-COMPRA ACCIONES,004-D055-FONDO DE PENSION APORT EMPL,004-D071-RETN APORT PATRON F DE A,004-D072-RETN APORT PATRON PLAN DE P,007-D010-PENSION ALIMENTICIA,010-D095-CREDITO INFONAVIT,011-D093-CREDITO FONACOT,OtrosPagos,002-D100-SUBSIDIO PARA EL EMPLEO
0,44xx8x7x-26xx-4767-9xx4-x4x96x293441,4.0,xxx9612234x9,HERMAN,601 - General de Ley Personas Morales,xxxx800930x40,ANTONIO,605 - Sueldos y Salarios e Ingresos Asimilados...,2024-05-10 10:36:17,2014.0,...,1692,0,0,1692,261,0,0,0,-,0
1,2xxxxx0x-1268-46xx-87x0-8xx32503x4xx,4.0,xxx9612234x9,HERMAN,601 - General de Ley Personas Morales,xxxx710915xx4,MARIO,605 - Sueldos y Salarios e Ingresos Asimilados...,2024-05-10 10:31:46,85.0,...,2147,0,8310,2147,13851,0,3766,0,-,0
2,x87917xx-6415-4xxx-x2x0-4x860xx94x6x,4.0,xxx9612234x9,HERMAN,601 - General de Ley Personas Morales,xxxx6902121x7,ARTURO,605 - Sueldos y Salarios e Ingresos Asimilados...,2024-05-10 10:33:46,2013.0,...,0,0,0,0,0,0,0,0,-,0
3,4x515810-8780-4x0x-927x-xx7x52x681x0,4.0,xxx9612234x9,HERMAN,601 - General de Ley Personas Morales,xxxx6902121x7,ARTURO,605 - Sueldos y Salarios e Ingresos Asimilados...,2024-05-10 10:36:17,2015.0,...,2147,0,2314,2147,3856,0,0,0,-,0
4,xx1x00xx-x4xx-1244-x826-xxx06163x7x9,4.0,xxx9612234x9,HERMAN,601 - General de Ley Personas Morales,xxxx740425xx1,ROAL,605 - Sueldos y Salarios e Ingresos Asimilados...,2024-05-10 10:36:18,2016.0,...,2147,0,0,2147,553,0,0,0,-,0


In [ ]:
# Definir la ruta de la carpeta
from pathlib import Path

output_path = Path.cwd() / 'reportes'
output_path.mkdir(parents=True, exist_ok=True)

grouped = df_base_fiscal.groupby(['rfcEmisor', 'rfcReceptor', 'nombreReceptor']).sum(numeric_only=True).reset_index()

numeric_cols = ['004-D005-APORT EMPL FONDO DE AHORRO', '004-D009-COMPRA ACCIONES', 
                '004-D055-FONDO DE PENSION APORT EMPL', '004-D071-RETN APORT PATRON F DE A',
                '004-D072-RETN APORT PATRON PLAN DE P', '007-D010-PENSION ALIMENTICIA',
                '010-D095-CREDITO INFONAVIT', '011-D093-CREDITO FONACOT', '002-D100-SUBSIDIO PARA EL EMPLEO']

output_data = []

for rfc_emisor, group in df_base_fiscal.groupby('rfcEmisor'):
    # Obtener los receptores para cada receptor
    receptores = group.groupby(['rfcReceptor', 'nombreReceptor'])[numeric_cols].sum().reset_index()
    
    for _, row in receptores.iterrows():
        total_row = {
            'uuid': '-',
            'rfcReceptor': row['rfcReceptor'],
            'nombreReceptor': row['nombreReceptor'],
            'total': row[numeric_cols].sum(),
            'subtotal': row[numeric_cols].sum()
        }
        output_data.append(total_row)
        
        # Información para cada receptor
        detalles = group[group['rfcReceptor'] == row['rfcReceptor']]
        for _, detalle in detalles.iterrows():
            detalle_row = {
                'uuid': detalle['uuid'],
                'rfcReceptor': detalle['rfcReceptor'],
                'nombreReceptor': detalle['nombreReceptor'],
                'total': detalle[numeric_cols].sum(),  
                'subtotal': detalle[numeric_cols].sum()
            }
            output_data.append(detalle_row)

output_df = pd.DataFrame(output_data)

# Crear archivo Excel
output_file_name = output_path / 'reporte_agrupado.xlsx'
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Reporte', index=False)

# Crear Wokbook
workbook = writer.book
worksheet = writer.sheets['Reporte']

# Estilos
money_format = workbook.add_format({'num_format': '$#,##0.00'})
gray_format = workbook.add_format({'bg_color': '#D3D3D3', 'bold': True})

# Aplicar formatos
worksheet.set_column('A:A', 36)  # UUID
worksheet.set_column('B:B', 18)  # rfcReceptor
worksheet.set_column('C:C', 25)  # nombreReceptor
worksheet.set_column('D:E', 15, money_format)  # total y subtotal

# Aplicar formato a filas de total
for row_num in range(1, len(output_df) + 1):
    if output_df.iloc[row_num - 1]['uuid'] == '-':
        worksheet.set_row(row_num, None, gray_format)

writer.close()

In [ ]:
# Definir la ruta de la carpeta

output_path = Path.cwd() / 'reportes'
output_path.mkdir(parents=True, exist_ok=True) # Crea la capeta de salida si no existe
output_file_name = output_path / 'reporte_agrupado2.xlsx'

workbook = xlsxwriter.Workbook(output_file_name)
worksheet = workbook.add_worksheet(name='Folios por RFC') # Establece el nombre de la hoja en la que estaremos trabajando

# Esta lista la uso para establecer estilos para los datos monetarios
numeric_cols = ['total','subtotal','totalMxn','subtotalMxn','004-D005-APORT EMPL FONDO DE AHORRO', '004-D009-COMPRA ACCIONES', 
                '004-D055-FONDO DE PENSION APORT EMPL', '004-D071-RETN APORT PATRON F DE A',
                '004-D072-RETN APORT PATRON PLAN DE P', '007-D010-PENSION ALIMENTICIA',
                '010-D095-CREDITO INFONAVIT', '011-D093-CREDITO FONACOT', '002-D100-SUBSIDIO PARA EL EMPLEO']

# Estilos para excel
header_format = workbook.add_format({
    'bold': True,
    # 'border': 1,
    'bg_color': '#D3D3D3',
    'align': 'center'
})


summary_format = workbook.add_format({
    'bold': True,
    # 'border': 1,
    'num_format': '$#,##0.00',
    'bg_color': '#F2F2F2'
})

float_summary_format = workbook.add_format({
    'bold': True,
    # 'border': 1,
    'num_format': '#,##0.00',
    'bg_color': '#F2F2F2',
    'align': 'right',
})


string_format = workbook.add_format({
    # 'border': 1,
})

# Formato de fecha y hora
date_format = workbook.add_format({
    # 'border': 1,
    'num_format': 'dd/mm/yyyy hh:mm:ss'
})

# Solo fecha:
date_only_format = workbook.add_format({
    # 'border': 1,
    'num_format': 'dd/mm/yyyy'  
})

money_format = workbook.add_format({
    # 'border': 1,
    'num_format': '$#,##0.00'
    })

int_format = workbook.add_format({
    # 'border': 1,
    'num_format': '0',
})

int_align_format = workbook.add_format({
    # 'border': 1,
    'num_format': '0',
    'align': 'right',
})

float_format = workbook.add_format({
    # 'border': 1,
    'num_format': '#,##0.00',
    'align': 'right',
})

# Escritura de headers
headers = df_base_fiscal.columns.tolist()

for col, header in enumerate(headers):
    worksheet.write(0, col, header, header_format)

# Este valor determina desde que fila quieres empezar a escribir los datos
# Geneneralmente el valor del row = 0 se asigna para las columnas
# pero puedes hacerlo en otra fila para crear banners con el logo de la empresa

row = 1

# Crear agrupamiento por emisor y receptor
df_base_fiscal.sort_values('rfcEmisor')
grouped = df_base_fiscal.groupby(['rfcEmisor', 'rfcReceptor', 'nombreReceptor'])
date_columns = ['fechaPago','fechaInicialPago','fechaFinalPago', 'fechaEmision']
no_numeric_columns = ['version','folio','versionNomina']

for (rfc_emisor, rfc_receptor, nombre_receptor), group in grouped:
    # Nivel 1 de agrupación
    for col, header in enumerate(headers):
        if header in no_numeric_columns:
            worksheet.write(row, col, '-', string_format)
        elif header == 'numDiasPagados':
            worksheet.write(row, col, group[header].sum().sum(),float_summary_format)
        elif header == 'rfcEmisor':
            worksheet.write(row, col, rfc_emisor, string_format)
        elif header == 'rfcReceptor':
            worksheet.write(row, col, rfc_receptor, string_format)
        elif header == 'nombreReceptor':
            worksheet.write(row, col, nombre_receptor, string_format)
        elif pd.api.types.is_numeric_dtype(df_base_fiscal[header]):
            worksheet.write(row, col, group[header].sum().sum(),summary_format)
        else:
            worksheet.write(row, col, '-', string_format)

    worksheet.set_row(row, None, None, {'level': 1, 'collapsed': True})
    row += 1
    
    # Nivel 2 de agrupación
    for _, record in group.iterrows():
        for col, header in enumerate(headers):
            if header in no_numeric_columns:
                worksheet.write(row, col, record[header], int_format)
            elif header == 'numDiasPagados':
                worksheet.write(row, col, record[header], float_format)
            elif header in date_columns:
                worksheet.write(row, col, record[header], date_only_format)
            elif pd.api.types.is_numeric_dtype(df_base_fiscal[header]):
                worksheet.write(row, col, record[header], money_format)
            else:
                worksheet.write(row, col, record[header], string_format)
        worksheet.set_row(row, None, None, {'level': 2, 'hidden': True})
        row += 1

# Ajuste de ancho de columnas
for i, header in enumerate(headers):
    if header == 'uuid':
        worksheet.set_column(i, i, 36)
    elif header == 'rfcReceptor' or header == 'rfcEmisor':
        worksheet.set_column(i, i, 18)
    elif header == 'nombreReceptor' or header == 'nombreEmisor':
        worksheet.set_column(i, i, 25)
    elif header in numeric_cols:
        worksheet.set_column(i, i, 25)
    else:
        worksheet.set_column(i, i, header.__len__())

# Para fijar columnas o "freeze" debes colocar el rango de indices de las columnas que prefieras, A = 0, B = 1 ... etc.
# Ejemplo: (A-E) = freeze_panes(0, 4)
# Para este ejemplo solo fije la columna principal (A,A)
worksheet.freeze_panes(0, 0)
worksheet.outline_settings(True, False, True, True)

workbook.close()